In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:

# Read text file containing the synthesis parameters
with open('g1_s1_beed_est_l.txt', 'r') as file:
    first_string = file.readline()  # the signal parameters
    succeeding_strings = file.readlines() # amplitude and frequency estimates

Fs = 44100

# Convert the strings to floats
tokens = first_string.split()
Nf = float(tokens[0]) # Nf is the number of frames (samples) per time slice
n = float(tokens[1]) # n is the number of time slices
dt = float(tokens[2]) # dt is the time interval between time slices

# Get the amplitude values and store in a list
an = []
for i in range(0, len(succeeding_strings), 2):
    result = []
    for value in succeeding_strings[i].split():
        result.append(float(value))
    an.append(result)

# Get the frequency values and store in a list
fn = []
for i in range(1, len(succeeding_strings), 2):
    result = []
    for value in succeeding_strings[i].split():
        result.append(float(value))
    fn.append(result)

an = np.array(an)
fn = np.array(fn)

In [3]:
def sos_interp(a, f, Fs, dt):
    """
        Interpolates the signal values of an input partial from its amplitude and frequency values
        Adapted from sosinterp.m by Franklin Agsaway, 2005
    """

    NperFrame = round(dt * Fs)      # number of samples per frame (original sample length per frame == 44100 * 0.015986 = 705 samples)
    dt = NperFrame / Fs
    Nf = len(a)     # number of frames 

    dA = np.append(np.diff(a), -a[-1])
    dA = dA / NperFrame     # amplitude increment per sample 
    # Amplitude synthesis interpolation
    asyn = np.zeros(Nf * NperFrame)    # Each row in Ats represents the interpolated amp values of each partial
    asyn[::NperFrame] = a
    for i in range(1, NperFrame):
        asyn[i::NperFrame] = asyn[(i-1)::NperFrame] + dA
    # Frequency synthesis interpolation
    fsyn = np.zeros(NperFrame * Nf)  # Each row in Fts represents the frame-varying freq values of each partial
    for i in range(NperFrame):
        fsyn[i::NperFrame] = f
    # Phase synthesis interpolation
    startphase = 0
    p = np.zeros(Nf)
    for i in range(Nf):
        p[i] = startphase - 2 * np.pi * ((i) * dt) * f[i]
        startphase += 2 * np.pi * dt * f[i]

    psyn = np.zeros(NperFrame * Nf)  # Each row in Fts represents the frame-varying freq values of each partial
    for i in range(NperFrame):
        psyn[i::NperFrame] = p

    t = np.arange(0, dt * Nf, 1 / Fs)
    sig = asyn * np.sin(2 * np.pi * fsyn * t + psyn)

    return sig



In [10]:
# sos
sig = np.zeros(71910) 
for i in range(1, len(an)):
    partial = sos_interp(an[i], fn[i], Fs, dt)
    sig = sig + partial

In [ ]:

# Time specifications
duration = 5.0 # Duration of the signal in seconds
sampling_rate = 441000

# Create an array of time values
t = np.linspace(0, duration, int(duration * sampling_rate), endpoint=False)

# Amplitude and frequency values for each sinusoid
amplitudes = []
frequencies = []

# Synthesize the signal by summing the sinusoids
signal = np.zeros_like(t)
for amp, freq in zip(amplitudes, frequencies):
    signal += amp * np.sin(2 * np.pi * freq * t)

# Plot the resulting waveform
plt.figure()
plt.plot(t, signal)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Synthesized Signal')
plt.show()